# Data Analytics Project

First, the necessary modules are imported.

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
from matplotlib import pyplot as plt 
import warnings 
warnings.simplefilter('ignore', category=UserWarning)

%matplotlib inline

Second, a DataFrame is created using the ecommerce data.

In [7]:
# The file containing the dataset uses a different encoding than the default 'utf-8', so this is specified
df = pd.read_csv('ecommerce.csv', encoding='cp1252') 

## Exploratory Data Analysis

There are 8 different columns present in the DataFrame.

In [8]:
df.columns # returns a list of columns in the dataframe

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [9]:
df.dtypes # returns the type for each column

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [10]:
df.shape # returns the number of rows (541909) and columns (8)

(541909, 8)

In [11]:
df.head() # returns the first five rows of the dataframe

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### Numerical Variables

Upon inspection of the description of the numerical values in the dataset, something is not right! The 'CustomerID' column should not be a continuous variable.

In [12]:
df.describe() # returns information about each numeric variable

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [22]:
print(f"CustomerID was dtype: {df['CustomerID'].dtype}")
df = df.astype({'CustomerID': str}) # converts CustomerID into a categorical variable
print(f"CustomerID is now dtype: {df['CustomerID'].dtype}")
df.info() # returns a list of all variables and their associated dtypes

CustomerID was dtype: object
CustomerID is now dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   541909 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


### Categorical Variables